# 記得要改桶子名子

In [19]:
import pandas as pd
import json
import requests
import pprint
from pymongo import MongoClient
from requests.adapters import HTTPAdapter

headers={'Connection': 'close'}
client = MongoClient("mongodb://127.0.0.1:27017")
db = client["fb"]
collection           = db["<db_name>"]            ###<<<<<<<<<<<<=============
political_reactions  = db["<collecion_name>"]     ###<<<<<<<<<<<<=============

In [25]:
def get_reactions(post_id):
    list_f = []
    ##first page
    resp = requests.get("https://graph.facebook.com/v2.5/%s?fields=reactions.limit(100){type,name}&access_token=<your fb token>"%post_id,headers=headers)
    
    ##first page insert to list
    for o in resp.json()["reactions"]["data"]:
        try:
            dic_1 = {}
            dic_1["post_id"] = post_id
            dic_1["id"]      = o["id"]
            dic_1["name"]    = o["name"]
            dic_1["type"]    = o["type"]
            list_f.append(dic_1)
        except:
            pass
    
    ##second page
    if 'next' in resp.json()['reactions']['paging']:
        p2 = resp.json()['reactions']['paging']['next']
    
    ##other page
        res = requests.get(p2)
        while "paging" in res.json():
            for k in res.json()["data"]:
                try:
                    dic_2 = {}
                    dic_2["post_id"] = post_id
                    dic_2["id"]      = k["id"]
                    dic_2["name"]    = k["name"]
                    dic_2["type"]    = k["type"]
                    list_f.append(dic_2)
                except:
                    pass

            if "next" in res.json()["paging"]:
                res = requests.get(res.json()["paging"]["next"])
            else:
                break
        return list_f
    else :
        return None

In [31]:
def main():
    pid_cursor = collection.find({},{"_id":0,"post_id":1},no_cursor_timeout = True)
    for post_id in pid_cursor:
#         post_id = post_id['post_id']
        react = get_reactions(post_id)
        if react:
            political_reactions.insert_many(react)
        print("finish : %s"%post_id)
    pid_cursor.close()

In [ ]:
if __name__ == "__main__":
    main()